In [1]:
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from sklearn import linear_model
import sklearn
import gzip
import os
from collections import defaultdict
import random
import pandas as pd
from sklearn import linear_model
import dateutil.parser
from datetime import timedelta
from tqdm import tqdm
import csv
import pickle
import numpy as np
import xgboost as xgb

In [2]:

# !pip install implicit
# !pip install surprise
# from implicit import bpr
# from surprise import SVD, Reader, Dataset
# from surprise.model_selection import train_test_split

path = "."
os.chdir(path)

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
#print(user_data[0])
#userName : string
#jobs : nested list [['Tổng công ty IDICO', 'Chuyên viên Kỹ thuật XD', [[1, 1, 1998], [1, 1, 2013], 1], '', '']]
#currentPlace : ['Thành phố Hồ Chí Minh, Việt Nam', [[], 108230990, 1066296640, 1]]
#previousPlaces : [['tp. Nam Định, Nam Định, Việt Nam', [[], 204200000, 1061683330, 1]]]
#education : nested list
#gPlusUserId : string

In [4]:
#print(places_data[0])
#attributes of places_data
#name : string
#price : None / 
#address : list(strings)
#hours : [['Monday', [['6:30 am--4:15 pm']], ['Tuesday', [['6:30 am--4:15 pm']]]] sometimes number like 1 is also there not sure why
#phone : string
#gPlusPlaceId : string
#gps: [float,float]

In [5]:
state_list = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
state_list = set(state_list)

In [6]:
'''
req_places = []
for d in places_data:
    t = d["address"][-1].split()
      for ad in t:
        if ad in state_list:
            req_places.append(d)
            '''

'\nreq_places = []\nfor d in places_data:\n    t = d["address"][-1].split()\n      for ad in t:\n        if ad in state_list:\n            req_places.append(d)\n            '

In [7]:
with open("USA_places.pkl","rb") as fp:
    req_places = pickle.load(fp)

In [8]:
#places_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/places.clean.json.gz"))
#user_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/users.clean.json.gz"))
#review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

print("done")

done


In [9]:
# review_data = list(parse("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/reviews.clean.json.gz"))

In [10]:
# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_1.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_2.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

# with open("/content/drive/MyDrive/UCSD_assignments/CS258/Assignment2/review_data_3.pkl","rb") as fp:
#   review_data1 = pickle.load(fp)

In [11]:
#review_data1[2000]

In [12]:
req_places[100000]

{'name': 'Office Depot',
 'price': '$$',
 'address': ['1406 W Expy 83', 'Weslaco, TX 78596'],
 'hours': [['Monday', [['8:00 am--9:00 pm']]],
  ['Tuesday', [['8:00 am--9:00 pm']]],
  ['Wednesday', [['8:00 am--9:00 pm']], 1],
  ['Thursday', [['8:00 am--9:00 pm']]],
  ['Friday', [['8:00 am--9:00 pm']]],
  ['Saturday', [['9:00 am--9:00 pm']]],
  ['Sunday', [['10:00 am--7:00 pm']]]],
 'phone': '(956) 968-6207',
 'closed': False,
 'gPlusPlaceId': '103560782244834035321',
 'gps': [26176207.0, -98024823.0]}

In [13]:
req_places[122789]
closed_places = []

In [14]:
# # with open("USA_places.pkl","wb") as fp:
# #   pickle.dump(req_places,fp)

# with open("USA_places.pkl","rb") as fp:
#     req_places = pickle.load(fp)

In [15]:
place_ids = [d['gPlusPlaceId'] for d in req_places]
place_id_set = set(place_ids)

In [16]:
dollar =set([d['price'] for d in req_places])
dollar

{'', '$', '$$', '$$$', None}

In [17]:
# review1_req = []
# for d in review_data1:
#     if d['gPlusPlaceId'] in place_id_set:
#         review1_req.append(d)

In [18]:
# with open("USA_reviews1.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews2.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews3.pkl","wb") as fp:
#   pickle.dump(review1_req,fp)

# with open("USA_reviews1.pkl","rb") as fp:
#     review1_req = pickle.load(fp)

# with open("USA_reviews2.pkl","rb") as fp:
#     review2_req = pickle.load(fp)

# with open("USA_reviews3.pkl","rb") as fp:
#     review3_req = pickle.load(fp)

In [19]:
# review1_req.extend(review2_req)
# review1_req.extend(review3_req)

In [20]:
# review1_req_indices = np.random.choice(len(review1_req), int(0.1*len(review1_req)))
# review1_req_set = set(review1_req_indices)
# review1_req_train = []
# review1_req_test = []
# for i in range(len(review1_req)):
#     if i not in review1_req_set:
#         review1_req_train.append(review1_req[i])
#     else:
#         review1_req_test.append(review1_req[i])

In [21]:
with open("USA_reviews_train.pkl","rb") as fp:
    review1_req_train = pickle.load(fp)
    
with open("USA_reviews_test.pkl","rb") as fp:
    review1_req_test = pickle.load(fp)


In [22]:
for d in review1_req_train:
    d['user'] = d['gPlusUserId']
    d['place'] = d['gPlusPlaceId']
    
for d in review1_req_test:
    d['user'] = d['gPlusUserId']
    d['place'] = d['gPlusPlaceId']

In [23]:
# with open("USA_reviews_train.pkl","wb") as fp:
#     pickle.dump(review1_req_train,fp)
    
# with open("USA_reviews_test.pkl","wb") as fp:
#     pickle.dump(review1_req_test,fp)


In [24]:
#filtering user data
# user_ids = set([d['gPlusUserId'] for d in review3_req])

In [25]:
# with open("review1_uids.pkl","rb") as fp:
#     user_ids1 = pickle.load(fp)
# with open("review2_uids.pkl","rb") as fp:
#     user_ids2 = pickle.load(fp)
# with open("review3_uids.pkl","rb") as fp:
#     user_ids3 = pickle.load(fp)

In [26]:
# all_user_ids = user_ids1.union(user_ids2).union(user_ids3)
with open("All_req_uids.pkl","rb") as fp:
    all_user_ids = pickle.load(fp)

In [27]:
# req_users_info = []
# for d in user_data:
#     if d['gPlusUserId'] in all_user_ids:
#         req_users_info.append(d)

In [28]:
# with open("User_Info_req.pkl","wb") as fp:
#     pickle.dump(req_users_info,fp)
with open("User_Info_req.pkl","rb") as fp:
    req_users = pickle.load(fp)


In [38]:
dates = []
for d in req_users[:10000]:
    values = [v for k,v in d.items()]
    if None not in values:
        dates.append(d['jobs'][0])


In [40]:
review1_req_train[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125',
 'user': '100000032416892623125',
 'place': '106591714648856494903'}

In [35]:
req_users[400]

{'userName': 'Chris Bupp',
 'jobs': None,
 'currentPlace': None,
 'previousPlaces': None,
 'education': [[[], [], [], [], [], 6],
  [['Mississippi State University',
    'Computer Science',
    [[1, 1, 2003], [1, 1, 2008]],
    'http://www.freebase.com/view/m/01dzg0',
    '']]],
 'gPlusUserId': '100003209077114320975'}

In [43]:
# data = [[d['gPlusUserId'],d['gPlusPlaceId'], d['rating']] for d in review1_req_train]
data = review1_req_train

In [53]:


# Utility data structures to stores Users per Item and Items Per User
usersPerPlace = defaultdict(set)
placesPerUser = defaultdict(set)

# Utility data structures to stores Ratings per Item and Ratings Per User
userRatings = defaultdict(dict)
placeRatings = defaultdict(dict)

# for user,recipe,rating in readCSV("trainInteractions.csv.gz"):
#     data.append({'user': user, 'recipe': recipe, 'rating':rating})
    
training_set_size = int(0.8*len(review1_req_train))
train = data[:training_set_size]
valid = data[training_set_size:]

for d in train:
    
    userRatings[d['user']][d['place']] = int(d['rating'])
    placeRatings[d['place']][d['user']] = int(d['rating'])
    usersPerPlace[d['place']].add(d['user'])
    placesPerUser[d['user']].add(d['place'])

In [54]:
# with open("usersPerPlace.pkl","wb") as fp:
#     pickle.dump(usersPerPlace,fp)

# with open("placesPerUser.pkl","wb") as fp:
#     pickle.dump(placesPerUser,fp)

# with open("userRatings.pkl","wb") as fp:
#     pickle.dump(userRatings,fp)

# with open("placeRatings.pkl","wb") as fp:
#     pickle.dump(placeRatings,fp)

In [45]:
allcategories = set()

for d in data:
    if d['categories']:
        for c in d['categories']:
            allcategories.add(c)

In [50]:
allcategories_list = list(allcategories)

In [51]:
[x for x in allcategories_list if len(x)<=2]

['大学',
 '89',
 '28',
 '4',
 '79',
 'MO',
 'U',
 '3',
 '92',
 'DJ',
 '8',
 '96',
 'It',
 '帯広',
 '12',
 '97',
 'Dj',
 'g',
 'RV']

In [ ]:
data[0]

In [ ]:
train = data[:int(len(data)*0.8)]

In [ ]:
# df = pd.DataFrame(data, columns=['user', 'place','rating'])

# reader = Reader(rating_scale=(0, 5))
# training_data = Dataset.load_from_df(df[['user', 'place', 'rating']], reader)

# trainset, validationset = train_test_split(training_data, shuffle = False, test_size = 0.20)

In [ ]:
# model = SVD(n_epochs= 15, lr_all= 0.005, n_factors= 1, reg_all= 2)    
# model.fit(trainset)

In [ ]:
# predictions_train = model.test(train)
# predictions_val = model.test(validationset)

In [ ]:
# sse = 0
# for p in predictions_train:
#     sse += (p.r_ui - p.est)**2

# print('Training set MSE is equal to ', sse / len(predictions_train))

# sse = 0
# for p in predictions_val:
#     sse += (p.r_ui - p.est)**2

# print('Validation set MSE is equal to ', sse / len(predictions_val))